In [ ]:
!pip install cherche --upgrade -q
!pip install ir_datasets -q
!pip install pytrec_eval
!pip install sentence_transformers

In [ ]:
import pytrec_eval
import ir_datasets
import pandas as pd
import json
from cherche import data, retrieve, rank
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csc_matrix
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csc_matrix
import typing
import ir_datasets



In [ ]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-7gwh4tai/python-terrier_dec6d14d330e4a56a7570fd826b907ff
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-7gwh4tai/python-terrier_dec6d14d330e4a56a7570fd826b907ff
  Resolved https://github.com/terrier-org/pyterrier.git to commit a17f9b7ed1782010b4680457874f220b65fa2594
  Preparing metadata (setup.py) ... done


In [ ]:
import pyterrier as pt
import ir_datasets

# Initialize PyTerrier
if not pt.started():
    pt.init()

# Load the vaswani dataset
dataset = ir_datasets.load('vaswani')

# Define a generator to yield documents in the format required by PyTerrier
def generate_documents():
    for doc in dataset.docs_iter():
        yield {'docno': doc.doc_id, 'text': doc.text}

# Convert the generator to a list
documents = list(generate_documents())

# Define the index directory
index_dir = './index/vaswani'

# Create the index
indexer = pt.IterDictIndexer(index_dir)
index_ref = indexer.index(documents)

# Load the index
index = pt.IndexFactory.of(index_ref)

# Check the number of documents
print(index.getCollectionStatistics().toString())


Number of documents: 11429
Number of terms: 7756
Number of postings: 224573
Number of fields: 1
Number of tokens: 271581
Field names: [text]
Positions:   false



In [ ]:
print(dataset)

Dataset(id='vaswani', provides=['docs', 'queries', 'qrels'])


**Retrieval**

In [33]:
# Set up the BM25 retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
for query in dataset.queries_iter():
    query_text = query.text
    query_id = query.query_id

    # Perform the retrieval
    results = bm25.search(query_text)

    # Print the top-ranked documents for the current query
    print(f"Query ID: {query_id}")
    print(f"Query Text: {query_text}")
    print(results)


Query ID: 1
Query Text: MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES

    qid  docid docno  rank      score  \
0     1   8171  8172     0  24.566031   
1     1   9880  9881     1  22.110514   
2     1   5501  5502     2  21.717148   
3     1   1501  1502     3  19.478355   
4     1   9858  9859     4  18.626342   
..   ..    ...   ...   ...        ...   
995   1      6     7   995   4.647515   
996   1     50    51   996   4.647515   
997   1   1028  1029   997   4.647515   
998   1   1536  1537   998   4.647515   
999   1   3042  3043   999   4.647515   

                                                 query  
0    MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...  
1    MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...  
2    MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...  
3    MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...  
4    MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...  
..                                                 ...  
99

In [37]:
class Ranker:

  def rank(self,index):
      TF_IDF =  pt.BatchRetrieve(index, wmodel= "TF_IDF")
      TF =  pt.BatchRetrieve(index, wmodel= "Tf")
      BM25 = pt.BatchRetrieve(index, wmodel= "BM25")
      pipeline = (TF_IDF | TF) >> BM25
      return pipeline


In [38]:
# réponse Q2
ranker = Ranker()
pipeline = ranker.rank(index_ref)
print(pipeline)

Compose(Union(BR(TF_IDF), BR(Tf)), BR(BM25))


In [44]:
for query in dataset.queries_iter():
    query_text = query.text
    query_id = query.query_id

    results = pipeline.search("Radiation")

    print(f"Query ID: {query_id}")
    print(f"Query Text: {query_text}")
    print(results)


Query ID: 1
Query Text: MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS BY THE USE OF MICROWAVE TECHNIQUES

    qid  docid docno  rank     score      query
0     1   4874  4875     0  7.320891  Radiation
1     1   1135  1136     1  7.238332  Radiation
2     1   8511  8512     2  7.075086  Radiation
3     1   2850  2851     3  6.972942  Radiation
4     1    860   861     4  6.828960  Radiation
..   ..    ...   ...   ...       ...        ...
601   1   8059  8060   601  2.112368  Radiation
602   1   8620  8621   602  2.112368  Radiation
603   1   4888  4889   603  2.058354  Radiation
604   1   6405  6406   604  1.942459  Radiation
605   1   2899  2900   605  1.496951  Radiation

[606 rows x 6 columns]
Query ID: 2
Query Text: MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WAVEGUIDE FED MICROWAVE RADIATIONS

    qid  docid docno  rank     score      query
0     1   4874  4875     0  7.320891  Radiation
1     1   1135  1136     1  7.238332  Radiation
2     1   8511  8512     2  7.075086  Radiatio